# Combinações das bandas `RGB`, exemplo para o `Landsat-8` e `Sentinel-2`

1)  [Landsat-8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2): 30 m
    *   `SR_B4:` 0.636-0.673 μm - Vermelho (R)
    *   `SR_B3:` 0.533-0.590 μm - Verde (G)
    *   `SR_B2:` 0.452-0.512 μm - Azul (B)


2) [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED): 10 m
    *   `B4:` 0.664 μm (S2A) / 0.665 μm (S2B) - Vermelho (R)
    *   `B3:` 0.560 μm (S2A) / 0.559 μm (S2B) - Verde (G)
    *   `B2:` 0.496 μm (S2A) / 0.492 μm (S2B) - Azul (B)

# Inicializando o GEE

In [ ]:
# importando o GEE e Geemap
import ee
import geemap

# inicializando GEE
geemap.ee_initialize(project='ee-enrique')

# Mapa: `Município de São Paulo`

Verificando região de estudo

In [ ]:
# define regiao
municipio_sp = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Sao Paulo'))

# criando um mapa interativo
Map = geemap.Map()

# centra o mapa na região
Map.centerObject(municipio_sp, zoom=10)

# adicionando basemap
Map.add_basemap('Esri.WorldImagery')

# contorno de Porto Alegre
style = {'color': 'yellow', 'fillColor': '00000000'}
Map.addLayer(municipio_sp.style(**style), {}, 'Município de São Paulo')

# exibe na tela
Map

Mapa para a RMSP

In [ ]:
#==============================================================================================================#
#                                       SHAPEFILE DO MUNICÍPIO DE PORTO ALEGRE
#==============================================================================================================#
municipio_sp = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Sao Paulo'))
estado_sp = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM1_NAME', 'Sao Paulo'))

#==============================================================================================================#
#                                             LANDSAT-8
#==============================================================================================================#
# aplica fator de escala
def fator_escala(img):

    # Aplica fatores de escala
    opticas = img.select('SR_B.*').multiply(2.75e-05).add(-0.2)
    termais = img.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Retorna imagem com bandas escaladas
    return (img.addBands(opticas, None, True)
               .addBands(termais, None, True)
               .copyProperties(img, ['system:time_start'])
               .set('date', img.date().format('YYYY-MM-dd')))

# dados: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
             .filterBounds(municipio_sp) \
             .filterDate('2024-01-01','2025-01-01') \
             .filter(ee.Filter.lt('CLOUD_COVER', 1)) \
             .sort('CLOUD_COVER', True) \
             .map(fator_escala)

#==============================================================================================================#
#                                               SENTINEL-2
#==============================================================================================================#
# dados: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
              .filterBounds(municipio_sp) \
              .filterDate('2024-01-01','2025-01-01') \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1)) \
              .sort('CLOUDY_PIXEL_PERCENTAGE', True) \
              .map(lambda img: img.multiply(0.0001).copyProperties(img, img.propertyNames()))

#==============================================================================================================#
#                                               MAPA INTERATIVO
#==============================================================================================================#
# parâmetros de visualização
param_vis_landsat_8 = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.03, 'max': 0.19}
param_vis_sentinel_2 = {'bands': ['B4','B3','B2'], 'min': 0.03, 'max': 0.19}

# monta painel com 2 mapas
geemap.linked_maps(rows = 1,
                   cols = 2,
                   height = "400px",
                   center = [-23.72, -46.66],
                   zoom = 10,
                   ee_objects = [landsat8.mean().clip(municipio_sp), sentinel2.mean().clip(municipio_sp)],
                   vis_params = [param_vis_landsat_8, param_vis_sentinel_2],
                   labels = ['RGB Landsat-8', 'RGB Sentinel-2'],
                   label_position = "topright")

Mapa com zoom

In [ ]:
#==============================================================================================================#
#                                       SHAPEFILE DO MUNICÍPIO DE PORTO ALEGRE
#==============================================================================================================#
municipio_sp = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Sao Paulo'))
estado_sp = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM1_NAME', 'Sao Paulo'))

#==============================================================================================================#
#                                             LANDSAT-8
#==============================================================================================================#
# aplica fator de escala
def fator_escala(img):

    # Aplica fatores de escala
    opticas = img.select('SR_B.*').multiply(2.75e-05).add(-0.2)
    termais = img.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Retorna imagem com bandas escaladas
    return (img.addBands(opticas, None, True)
               .addBands(termais, None, True)
               .copyProperties(img, ['system:time_start'])
               .set('date', img.date().format('YYYY-MM-dd')))

# dados: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
             .filterBounds(estado_sp) \
             .filterDate('2024-01-01','2025-01-01') \
             .filter(ee.Filter.lt('CLOUD_COVER', 1)) \
             .sort('CLOUD_COVER', True) \
             .map(fator_escala)

#==============================================================================================================#
#                                               SENTINEL-2
#==============================================================================================================#
# dados: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
              .filterBounds(estado_sp) \
              .filterDate('2024-01-01','2025-01-01') \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1)) \
              .sort('CLOUDY_PIXEL_PERCENTAGE', True) \
              .map(lambda img: img.multiply(0.0001).copyProperties(img, img.propertyNames()))

#==============================================================================================================#
#                                               MAPA INTERATIVO
#==============================================================================================================#
# parâmetros de visualização
param_vis_landsat_8 = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.03, 'max': 0.19}
param_vis_sentinel_2 = {'bands': ['B4','B3','B2'], 'min': 0.03, 'max': 0.19}

# monta painel com 2 mapas
geemap.linked_maps(rows = 1,
                   cols = 2,
                   height = "400px",
                   center = [-23.72, -46.66],
                   zoom = 14,
                   ee_objects = [landsat8.mean(), sentinel2.mean()],
                   vis_params = [param_vis_landsat_8, param_vis_sentinel_2],
                   labels = ['RGB Landsat-8', 'RGB Sentinel-2'],
                   label_position = "topright")

In [ ]:
# mostrando os dados do Landsat-8
landsat8

In [ ]:
# mostrando os dados do Sentinel-2
sentinel2

In [ ]:
# informações de uma imagem do Landsat-8
landsat8.first().getInfo()

In [ ]:
# informações de uma imagem do Sentinel-2
sentinel2.first().getInfo()